<a href="https://colab.research.google.com/github/aliikhwan99/LLM_MODEL/blob/main/model_Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Call Mallam model

In [ ]:
!pip install -U transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 75.6 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Create pipeline
pipe = pipeline("text-generation", model="mesolitica/mallam-1.1B-4096")

# Generate text
output = pipe("Apa khabar?", max_length=100, do_sample=True)
print(output[0]['generated_text'])

# Alternatively, use model and tokenizer directly
tokenizer = AutoTokenizer.from_pretrained("mesolitica/mallam-1.1B-4096")
model = AutoModelForCausalLM.from_pretrained("mesolitica/mallam-1.1B-4096")

inputs = tokenizer("Apa khabar?", return_tensors="pt")
outputs = model.generate(**inputs, max_length=100, do_sample=True)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


2025-04-20 03:08:15.658218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745118496.101307      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745118496.222438      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Apa khabar?\nAnda ada soalan nak tanya? Boleh terus tanya di sini. Mana tahu kalau saya boleh bantu menjawab dan kita kongsi ilmu bersama.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Apa khabar? Bagaimana rasa madu? Bagaimana rasanya?


# Evaluation for translation task

# Step 1: Prepare a Translation Dataset

In [ ]:
test_data = [
    {"input": "Hello, how are you?", "expected": "Hai, apa khabar?"},
    {"input": "I love to eat nasi lemak.", "expected": "Saya suka makan nasi lemak."},
    {"input": "Where is the nearest hospital?", "expected": "Di manakah hospital yang terdekat?"},
]


# Step 2: Prompt the Model to Translate

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="mesolitica/mallam-1.1B-4096")

results = []
for sample in test_data:
    prompt = f"Translate to Malay: {sample['input']}"
    output = pipe(prompt, max_length=100, do_sample=False)[0]['generated_text']
    results.append({
        "input": sample["input"],
        "expected": sample["expected"],
        "output": output.strip()
    })


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


#  Step 3: Evaluate the Output

In [ ]:
!pip install evaluate
import evaluate

bleu = evaluate.load("bleu")

predictions = [r["output"] for r in results]
references = [[r["expected"]] for r in results]  # Note: nested list for BLEU

score = bleu.compute(predictions=predictions, references=references)
print(f"BLEU Score: {score['bleu']:.4f}")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu

BLEU Score: 0.0000


In [ ]:
for r in results:
    print(f"\nInput: {r['input']}")
    print(f"Expected: {r['expected']}")
    print(f"Model Output: {r['output']}")



Input: Hello, how are you?
Expected: Hai, apa khabar?
Model Output: Translate to Malay: Hello, how are you? I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine.

Input: I love to eat nasi lemak.
Expected: Saya suka makan nasi lemak.
Model Output: Translate to Malay: I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat

Input: Where is the nearest hospital?
Expected: Di manakah hospital yang terdekat?
Model Output: Translate to Malay: Where is the nearest hospital?\nPosted by Mohd.Noor Azam bin Atan at 10:00 AM No comments:\nPos

# Step-by-Step: Access Layers, Attention, and Model Internals

# Load the model manually (not with pipeline)

# Inspect Model Architecture Without Weights

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("mesolitica/mallam-1.1B-4096")

print(config)


# Stack SAT on Mallam

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoConfig, AutoModelForCausalLM

# Load Mallam model config only (no weights)
config = AutoConfig.from_pretrained("mesolitica/mallam-1.1B-4096")
mallam_base = AutoModelForCausalLM.from_config(config)  # No pre-trained weights loaded


# STA Structure

In [ ]:
class SyntaxAwareAttention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, hidden_size)

    def forward(self, hidden_states, syntax_mask=None):
        # 🔥 Convert hidden states to float32 to avoid dtype mismatch
        hidden_states = hidden_states.to(torch.float32)

        Q = self.query(hidden_states)
        K = self.key(hidden_states)
        V = self.value(hidden_states)

        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / (Q.size(-1) ** 0.5)

        if syntax_mask is not None:
            attn_scores = attn_scores.masked_fill(syntax_mask == 0, -1e9)

        attn_probs = torch.softmax(attn_scores, dim=-1)
        context = torch.matmul(attn_probs, V)
        output = self.out(context)
        return output


# Wrap Everything Into One Model

In [ ]:
def forward(self, input_ids, attention_mask=None, syntax_mask=None, **kwargs):
    # Get hidden states from base model
    outputs = self.base_model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        output_hidden_states=True,
        **kwargs
    )
    hidden = outputs.hidden_states[-1]  # Shape: [batch, seq_len, hidden_size]

    # Apply Syntax-Aware Attention
    syntax_out = self.syntax_attn(hidden, syntax_mask)
    combined = hidden + syntax_out  # Simple residual connection

    return combined  # or return logits if needed


#  Initialize Model

In [ ]:
# Initialize stacked model
model = MallamWithSAT(mallam_base, hidden_size=config.hidden_size)

# Example dummy input
batch_size = 2
seq_len = 16
dummy_input = torch.randint(0, config.vocab_size, (batch_size, seq_len))
dummy_mask = torch.ones((batch_size, seq_len), dtype=torch.int64)
dummy_syntax_mask = torch.ones((batch_size, seq_len, seq_len))  # You can mask based on syntax tree later

# Forward pass (testing architecture)
with torch.no_grad():
    out = model(input_ids=dummy_input, attention_mask=dummy_mask, syntax_mask=dummy_syntax_mask)
    print(out.shape)  # Expect: [batch_size, seq_len, hidden_size]


torch.Size([2, 16, 2048])


# Use Pretrained Weights Later (Optional)

In [ ]:
# If you want to load full weights later
from transformers import AutoModelForCausalLM
mallam_with_weights = AutoModelForCausalLM.from_pretrained("mesolitica/mallam-1.1B-4096")
model = MallamWithSAT(mallam_with_weights, hidden_size=config.hidden_size)


# Evaluation

In [ ]:
import torch
import torch.nn as nn
import time

def compare_models(base_model, sat_model, input_ids, attention_mask, syntax_mask=None):
    base_model.eval()
    sat_model.eval()

    with torch.no_grad():
        # Inference with base Mallam
        start = time.time()
        base_output = base_model(input_ids=input_ids, attention_mask=attention_mask)
        base_time = time.time() - start

        # Get logits or hidden state depending on model's output
        base_hidden = base_output.logits if hasattr(base_output, 'logits') else base_output[0]

        # Inference with Mallam + SAT
        start = time.time()
        sat_output = sat_model(input_ids=input_ids, attention_mask=attention_mask, syntax_mask=syntax_mask)
        sat_time = time.time() - start

        # Output results
        print(f"[Mallam] Output shape: {base_hidden.shape}, Time: {base_time:.4f}s")
        print(f"[Mallam + SAT] Output shape: {sat_output.shape}, Time: {sat_time:.4f}s")

        # Optional: calculate diff
        diff = torch.abs(base_hidden - sat_output).mean().item()
        print(f"Mean absolute difference in outputs: {diff:.6f}")


In [ ]:
# Create dummy inputs
batch_size = 2
seq_len = 16
vocab_size = config.vocab_size  # from your config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dummy_input_ids = torch.randint(0, vocab_size, (batch_size, seq_len)).to(device)
dummy_attention_mask = torch.ones((batch_size, seq_len), dtype=torch.int64).to(device)
dummy_syntax_mask = torch.ones((batch_size, seq_len, seq_len)).to(device)  # or a real syntax mask


In [ ]:
def forward(self, input_ids, attention_mask=None, syntax_mask=None):
    # Step 1: Get base model hidden states (not logits yet)
    base_output = self.base_model.model(
        input_ids=input_ids,
        attention_mask=attention_mask
    )
    hidden_states = base_output.last_hidden_state  # likely bfloat16

    # Step 2: Convert to float32 before passing to SAT
    sat_input = hidden_states.to(dtype=torch.float32)

    # Step 3: Apply Syntax-Aware Attention
    modified_hidden = self.sat(sat_input, syntax_mask=syntax_mask)

    # Step 4: Convert back to lm_head's dtype (usually bfloat16)
    target_dtype = self.base_model.lm_head.weight.dtype
    modified_hidden = modified_hidden.to(dtype=target_dtype)

    # Step 5: Get final logits
    logits = self.base_model.lm_head(modified_hidden)
    return logits


In [ ]:
compare_models(mallam_base, model_with_sat, dummy_input_ids, dummy_attention_mask, dummy_syntax_mask)


RuntimeError: expected mat1 and mat2 to have the same dtype, but got: float != c10::BFloat16